In [1]:
def display_final_review(mistake_count, mistake_log):
    """Summarize the conversation and review user mistakes."""

    print("\n📊 🔥 Final Review:")

    if mistake_count == 0:
        print("✅ Well done! You made no mistakes. Keep practicing! 🎯")
    else:
        print(f"❗ You made {mistake_count} mistake(s) during the conversation.")
        
        # Categorizing mistakes (basic pattern detection)
        common_issues = {}
        
        for mistake in mistake_log:
            issue = mistake["bot_response"].split(".")[0]  # Extract first sentence as a pattern
            if issue in common_issues:
                common_issues[issue] += 1
            else:
                common_issues[issue] = 1

        # Display the most frequent mistakes
        print("\n🔎 **Common Mistake Patterns:**")
        for issue, count in common_issues.items():
            print(f"- {issue} (Occurred {count} time(s))")

        # Suggestions for improvement
        print("\n💡 **Suggestions for Improvement:**")
        if "grammar" in str(common_issues).lower():
            print("- Focus on grammar rules, especially tense and sentence structure.")
        if "vocabulary" in str(common_issues).lower():
            print("- Improve your vocabulary by learning new words and their usage.")
        if "pronunciation" in str(common_issues).lower():
            print("- Practice pronunciation with phonetic exercises.")

        print("\n🔥 Keep practicing to reduce mistakes! 💪")



In [ ]:
# chatbot.py

import time
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain_community.chat_models import ChatOpenAI
import traceback
import os
from db_manager import initialize_db, store_response, display_chat_history

# 🔥 Load environment variables
os.environ['OPENAI_API_KEY'] = "REPLACE WITH YOUR API KEY"

# 🔥 Initialize database
initialize_db()

# 🔥 Set up the chatbot model
def get_chatbot_model():
    """Initialize the language model."""
    try:
        return ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7)
    except Exception as e:
        print(f"Error initializing model: {e}")
        traceback.print_exc()


# 🔥 Conversation template
prompt_template = PromptTemplate(
    input_variables=["user", "known_lang", "learning_lang", "level", "user_input"],
    template="""
    You are a language tutor helping {user} learn {learning_lang}.
    The user speaks {known_lang} and is at {level} level.
    User says: {user_input}
    Respond with helpful sentences, corrections, and translations.
    """
)

# 🔥 Conversation loop
def chatbot_conversation():
    """Chat loop with SQLite integration and final review."""
    
    print("\n🛠️ Chatbot is ready! Type 'quit' to exit.\n")
    
    # Collect user info
    user = input("Enter your name: ").strip()
    known_lang = input("Language you know: ").strip()
    learning_lang = input("Language to learn: ").strip()
    level = input("Your level (Beginner, Intermediate, Advanced): ").strip()

    # The chatbot uses the GPT model and prompt template to generate responses.
    # The LLMChain takes user input and context to create a response.

    model = get_chatbot_model()
    chain = LLMChain(llm=model, prompt=prompt_template)

    mistake_count = 0
    mistake_log = []

    while True:
        try:
            user_input = input("\nYou: ").strip()

            if user_input.lower() in ('quit', 'q'):
                break

            # Prepare input data
            inputs = {
                "user": user,
                "known_lang": known_lang,
                "learning_lang": learning_lang,
                "level": level,
                "user_input": user_input
            }
            ######print("input taken")
            # Get response
            response = chain.run(inputs)
            ######print("input response taken")
            # 🔥 Mistake Detection Logic (Enhanced)
            mistake_keywords = [
                "incorrect", "wrong",
                "the correct sentence is", "you should say",
                "correction:", "you can say", "Let me help you","correct phrase"
            ]
            ######print("mistake keywords validated")
            mistake_detected = any(
                keyword.lower() in response.lower() for keyword in mistake_keywords
            )

            # Store in database
            store_response(user_input, response, 1 if mistake_detected else 0)
            ######print("response stored")
            # Log mistakes
            if mistake_detected:
                mistake_count += 1
                mistake_log.append(response)

            # Display conversation
            print(f"\n🤖 [Chatbot]: {response}")
            if mistake_detected:
                print("❗ Mistake detected ✅")
        except Exception as e:
            print(f"\nError: {e}")
            traceback.print_exc()

    # Display chat history and final review
    display_chat_history()
    display_final_review(mistake_count, mistake_log)


# 🔥 Main function
if __name__ == "__main__":
    chatbot_conversation()



🛠️ Chatbot is ready! Type 'quit' to exit.



Enter your name:  sdf
Language you know:  english
Language to learn:  hindi
Your level (Beginner, Intermediate, Advanced):  beginner

You:  hi



🤖 [Chatbot]: Hello! In Hindi, you can greet someone by saying "नमस्ते" (namaste). It's a common way to say hi in Hindi. Keep practicing!
